## ANÁLISE DE ATENDIMENTOS PELO SP156

O SP156 é um canal de comunicação da Prefeitura de São Paulo pelo qual cidadãos podem fazer solicitações e reclamações. Mais informações podem ser encontradas em https://sp156.prefeitura.sp.gov.br/.

A análise busca investigar quantos registros ainda não foram atendidos, quais os locais das ocorrências e quais os tipos mais comuns de solicitações. Também busca fazer um comparativo entre o primeiro semestre de 2016 (gestão Haddad) e o primeiro semestre de 2017 (gestão Doria).

Os dados utilizados foram obtidos pelo portal Dados Abertos, em http://dados.prefeitura.sp.gov.br/.

**A análise a seguir não está completa.** O estudo está sendo executado aos poucos. Portanto, os resultados serão apresentados ao longo dos próximos dias.

In [1]:
# Importação das ferramentas de análise
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# Seleção dos dados utilizados na análise
dataset1s2016 = pd.read_csv('data/sp156solicitacoes-detalhadas1sem2016.csv', 
                            sep=';', 
                            encoding='latin-1',
                            low_memory=False)
dataset2s2016 = pd.read_csv('data/sp156solicitacoes-detalhadas2sem2016.csv', 
                            sep=';', 
                            encoding='latin-1',
                            low_memory=False)
dataset1s2017 = pd.read_csv('data/sp156solicitacoes-detalhadas1sem2017.csv', 
                            sep=';', 
                            encoding='latin-1',
                            low_memory=False)
# Concatenação dos dados
dataset = pd.concat([dataset1s2016, 
                     dataset2s2016, 
                     dataset1s2017])

# Mensuração dos registros em análise
print("Quantidade de registros em análise")
dataset.shape

Quantidade de registros em análise


(1150164, 11)

In [2]:
# Observação de como eles se distribuem
dataset.head(10)

,Data de Abertura,Hora Abertura,Prefeitura Regional,Distrito,Órgão,Tema,Assunto,Serviço,Status da Solicitação,Data do Parecer,Hora do Parecer
0,01/01/2016,00:04,MOOCA,TATUAPE,SPTRANS,Transporte,"Conduta de trabalho do motorista, cobrador e f...",Denúncia de direção inadequada/perigosa de veí...,FINALIZADO,04/01/2016,16:36
1,01/01/2016,00:16,SAO MIGUEL,SAO MIGUEL,AMLURB,Lixo e limpeza,Conduta de funcionários,Denúncia de má conduta dos funcionários (motor...,FINALIZADO,05/01/2016,17:31
2,01/01/2016,00:20,LAPA,LAPA,SPTRANS,Transporte,Linhas e itinerários de ônibus,Alteração de itinerário ou ponto inicial/final,FINALIZADO,05/01/2016,11:58
3,01/01/2016,00:20,SE,REPUBLICA,SMADS,Cidadania e assistência social,População ou pessoa em situação de rua,Abordagem social a pessoa em situação de rua,FINALIZADO,01/01/2016,00:20
4,01/01/2016,00:21,ITAQUERA,CIDADE LIDER,CET,Trânsito,Circulação de veículos,Avaliação da circulação e acesso de veículos,FINALIZADO,07/06/2016,18:42
5,01/01/2016,00:28,PIRITUBA-JARAGUA,JARAGUA,SMPR,Rua e bairro,Buraco e pavimentação,Tapa-buraco,FINALIZADO,12/01/2016,02:00
6,01/01/2016,00:31,NaN,NaN,SMADS,Cidadania e assistência social,População ou pessoa em situação de rua,Abordagem social a pessoa em situação de rua,FINALIZADO,01/01/2016,00:31
7,01/01/2016,00:36,PIRITUBA-JARAGUA,JARAGUA,SMPR,Rua e bairro,Buraco e pavimentação,Tapa-buraco,FINALIZADO,12/01/2016,02:00
8,01/01/2016,00:36,CAPELA DO SOCORRO,CIDADE DUTRA,SMPR,Meio Ambiente,Árvore,Plantio de árvore em vias e áreas públicas,FINALIZADO,22/05/2017,17:17
9,01/01/2016,00:41,SE,SANTA CECILIA,SMADS,Cidadania e assistência social,População ou pessoa em situação de rua,Abordagem social a pessoa em situação de rua,FINALIZADO,01/01/2016,00:41


In [3]:
# Separação de dias, meses e anos
dataset['Data de Abertura'] = pd.to_datetime(dataset['Data de Abertura'], errors='coerce')
dataset['Dia de Abertura'] = dataset['Data de Abertura'].apply(lambda date: date.day)
dataset['Mês de Abertura'] = dataset['Data de Abertura'].apply(lambda date: date.month)
dataset['Ano de Abertura'] = dataset['Data de Abertura'].apply(lambda date: date.year)

In [4]:
# Contagem por status
dataset['Status da Solicitação'].value_counts()

FINALIZADO                691902
AGUARDANDO ATENDIMENTO    403639
CANCELADA                  54623
Name: Status da Solicitação, dtype: int64

In [5]:
# Separação por ano e por status
dataset_per_year = dataset.groupby(['Ano de Abertura'])
dataset_per_year['Status da Solicitação'].value_counts()

Ano de Abertura  Status da Solicitação 
2016             FINALIZADO                535736
                 AGUARDANDO ATENDIMENTO    189849
                 CANCELADA                  53178
2017             AGUARDANDO ATENDIMENTO    213790
                 FINALIZADO                156166
                 CANCELADA                   1445
Name: Status da Solicitação, dtype: int64

In [12]:
dataset_finalizado = dataset[dataset['Status da Solicitação'] == 'FINALIZADO']

In [13]:
dataset_finalizado.head()

,Data de Abertura,Hora Abertura,Prefeitura Regional,Distrito,Órgão,Tema,Assunto,Serviço,Status da Solicitação,Data do Parecer,Hora do Parecer,Dia de Abertura,Mês de Abertura,Ano de Abertura
0,2016-01-01,00:04,MOOCA,TATUAPE,SPTRANS,Transporte,"Conduta de trabalho do motorista, cobrador e f...",Denúncia de direção inadequada/perigosa de veí...,FINALIZADO,04/01/2016,16:36,1,1,2016
1,2016-01-01,00:16,SAO MIGUEL,SAO MIGUEL,AMLURB,Lixo e limpeza,Conduta de funcionários,Denúncia de má conduta dos funcionários (motor...,FINALIZADO,05/01/2016,17:31,1,1,2016
2,2016-01-01,00:20,LAPA,LAPA,SPTRANS,Transporte,Linhas e itinerários de ônibus,Alteração de itinerário ou ponto inicial/final,FINALIZADO,05/01/2016,11:58,1,1,2016
3,2016-01-01,00:20,SE,REPUBLICA,SMADS,Cidadania e assistência social,População ou pessoa em situação de rua,Abordagem social a pessoa em situação de rua,FINALIZADO,01/01/2016,00:20,1,1,2016
4,2016-01-01,00:21,ITAQUERA,CIDADE LIDER,CET,Trânsito,Circulação de veículos,Avaliação da circulação e acesso de veículos,FINALIZADO,07/06/2016,18:42,1,1,2016


In [14]:
# Separação de dias, meses e anos
dataset_finalizado['Data do Parecer'] = pd.to_datetime(dataset_finalizado['Data do Parecer'], errors='coerce')
dataset_finalizado['Dia do Parecer'] = dataset_finalizado['Data do Parecer'].apply(lambda date: date.day)
dataset_finalizado['Mês do Parecer'] = dataset_finalizado['Data do Parecer'].apply(lambda date: date.month)
dataset_finalizado['Ano do Parecer'] = dataset_finalizado['Data do Parecer'].apply(lambda date: date.year)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [16]:
dataset_finalizado.shape

(691902, 17)

In [31]:
dataset_finalizado_1s2016 = dataset_finalizado[dataset_finalizado['Ano do Parecer'] == 2016]
dataset_finalizado_1s2016 = dataset_finalizado_1s2016[dataset_finalizado_1s2016['Mês do Parecer'] <= 6]
dataset_finalizado_1s2016['Semestre'] = 1
dataset_finalizado_2s2016 = dataset_finalizado[dataset_finalizado['Ano do Parecer'] == 2016]
dataset_finalizado_2s2016 = dataset_finalizado_2s2016[dataset_finalizado_2s2016['Mês do Parecer'] >= 7]
dataset_finalizado_2s2016['Semestre'] = 2
dataset_finalizado_1s2017 = dataset_finalizado[dataset_finalizado['Ano do Parecer'] == 2017]
dataset_finalizado_1s2017 = dataset_finalizado_1s2017[dataset_finalizado_1s2017['Mês do Parecer'] <= 6]
dataset_finalizado_1s2017['Semestre'] = 1

In [32]:
dataset_finalizado_1s2016.shape

(245141, 18)

In [33]:
dataset_finalizado_2s2016.shape

(216215, 18)

In [34]:
dataset_finalizado_1s2017.shape

(175621, 18)

### Conclusão 1

Entre janeiro de 2016 e maio de 2017 -- último mês disponível nos dados da Prefeitura abertos ao público --, a SP156 registrou 1.150.164 ocorrências. Dessas, 691.902 foram finalizadas.

**Gestão Haddad**

O primeiro semestre de 2016 teve o maior número de ocorrências encerradas. Foram 245.141.
Houve uma queda no segundo semestre do mesmo ano, para 216.215 -- 11,79% em relação ao período anterior.

**Gestão Doria**

Até maio de 2017, a resolução de atendimentos teve nova diminuição. Foram 175.621, o que representa queda de 28,35% na comparação com o primeiro semestre de 2016, e queda de 18,77% em relação ao segundo semestre do ano passado.